In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_score, recall_score, f1_score

# Define the number of classes
num_classes = 2

# Load the image dataset using ImageDataGenerator
data_dir = 'preprocessed'
image_size = (224, 224)
batch_size = 32

datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)
print("------------------------")

validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)
print("------------------------")

# Define the ResNet-50 model
base_model = ResNet50(
    weights='imagenet',
    include_top=False,
    input_shape=(image_size[0], image_size[1], 3),
)
print("------------------------")

# Add custom classification layers
model = keras.models.Sequential([
    base_model,
    keras.layers.Flatten(),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(num_classes, activation='softmax')
])
print("------------------------")


# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
print("------------------------")

# Define 5-fold cross-validation
num_folds = 5
skf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)
print("------------------------")

precision_scores, recall_scores, f1_scores = [], [], []

# Perform 5-fold cross-validation
for train_index, val_index in skf.split(train_generator.filenames, train_generator.classes):
    train_data = train_generator
    val_data = validation_generator

    model.fit(train_data, epochs=5)  # Adjust the number of epochs as needed
    print("------------------------")

    # Predict on the validation data
    y_pred = model.predict(val_data)
    y_true = val_data.classes

    precision = precision_score(y_true, y_pred.argmax(axis=1), zero_division=0)
    recall = recall_score(y_true, y_pred.argmax(axis=1), zero_division=0)
    f1 = f1_score(y_true, y_pred.argmax(axis=1), zero_division=0)


    precision_scores.append(precision)
    recall_scores.append(recall)
    f1_scores.append(f1)

# Compute the average precision, recall, and F1 score
avg_precision = sum(precision_scores) / len(precision_scores)
avg_recall = sum(recall_scores) / len(recall_scores)
avg_f1 = sum(f1_scores) / len(f1_scores)

print(f'Average Precision: {avg_precision}')
print(f'Average Recall: {avg_recall}')
print(f'Average F1 Score: {avg_f1}')


Found 598 images belonging to 2 classes.
------------------------
Found 148 images belonging to 2 classes.
------------------------
------------------------
------------------------
------------------------
------------------------
Epoch 1/5
